<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

# MLRun Functions : The basic building blocks of Iguazio MLOps

### Initialize your MLRun environment

In [1]:
from mlrun import get_or_create_project

project = get_or_create_project("default", "./")

> 2023-05-09 10:26:09,882 [info] Server and client versions are not the same: {'parsed_server_version': VersionInfo(major=1, minor=2, patch=1, prerelease=None, build=None), 'parsed_client_version': VersionInfo(major=1, minor=1, patch=0, prerelease=None, build=None)}
> 2023-05-09 10:26:10,669 [info] loaded project default from MLRun DB


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Write your function code

In [2]:
# mlrun: start-code

import mlrun

def prep_data(context, source_url: mlrun.DataItem, label_column='label'):

    # Convert the DataItem to a pandas DataFrame
    df = source_url.as_df()
    df[label_column] = df[label_column].astype('category').cat.codes    
    
    # Log the DataFrame size after the run
    context.log_result('num_rows', df.shape[0])

    # Store the dataset in your artifacts database
    context.log_dataset('cleaned_data', df=df, index=False, format='csv', stats=True)

# mlrun: end-code

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### code_to_function - create an MLRun function from code

In [3]:
from mlrun import code_to_function

myfunc = code_to_function(name="prep-func",
                          project="default",
                          filename="mlrun_functions.ipynb", # prep_data.py
                          kind="job", # spark , dask, nuclio, serving(model server)
                          handler="prep_data",
                          image="mlrun/mlrun")

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Run the function locally

In [4]:
from os import path

source_url = 'https://s3.wasabisys.com/iguazio/data/iris/iris.data.raw.csv'
path = path.abspath('./')

myfunc.run(name="prep-func",
           handler="prep_data",
           inputs={'source_url': source_url},
           local=True,
           artifact_path=path)

> 2023-05-09 10:26:12,952 [info] starting run prep-func uid=8576f198755f4cbf89faafa2b725d693 DB=https://mlrun-api.default-tenant.app.us-sales-350.iguazio-cd1.com


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...b725d693,0,May 09 14:26:15,completed,prep-func,v3io_user=brennankind=owner=brennanhost=iguazio.local,source_url,,num_rows=150,cleaned_data


> 2023-05-09 10:26:16,858 [info] run executed, status=completed


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

### Run the function on the cluster

In [5]:
myfunc.run(name="prep-func",
           handler="prep_data",
           inputs={'source_url': source_url},
           local=False
           )

> 2023-05-09 10:26:16,878 [info] starting run prep-func uid=cb5c4cca1a154605b7487f3dee07e267 DB=https://mlrun-api.default-tenant.app.us-sales-350.iguazio-cd1.com
> 2023-05-09 10:26:17,426 [info] Job is running in the background, pod: prep-func-l4z72
> 2023-05-09 14:26:23,444 [info] Server and client versions are not the same: {'parsed_server_version': VersionInfo(major=1, minor=2, patch=1, prerelease=None, build=None), 'parsed_client_version': VersionInfo(major=1, minor=1, patch=0, prerelease=None, build=None)}
> 2023-05-09 14:26:25,177 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
default,...ee07e267,0,May 09 14:26:24,completed,prep-func,v3io_user=brennankind=jobowner=brennanmlrun/client_version=1.1.0host=prep-func-l4z72,source_url,,num_rows=150,cleaned_data


> 2023-05-09 10:26:27,960 [info] run executed, status=completed


<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

## Many ways to create a function

In [6]:
from mlrun import new_function, code_to_function, import_function

fx = new_function('some-func', kind='job', image='mlrun/mlrun', command='./myfunc.py')
fx = code_to_function() # convenience function of new_function, used in notebooks
fx = import_function("db://mlrun-functions/prep-func:latest")
fx = import_function("hub://sklearn_classifier")
fx = import_function("./func.yaml")
fx = import_function("https://raw.githubusercontent.com/org/repo/func.yaml")

fx.save(tag="1.4") # save to MLRun database
fx.export("./myfunc.yaml") # export function definition to a yaml file
fx.deploy() # build image w/ required packages, write to docker, deploy to service if online runtime
fx.run() # execute batch function
fx.as_step() # convert function to Kubeflow pipeline step
print(fx.to_yaml()) # view function yaml

# kind = local, job, nuclio, serving, dask, mpijob, spark

ValueError: please specify the function kind